层次化索引 hierarchical indexing .可以在一个轴上拥有多个(两个以上)索引级别,即以低纬度形式处理高纬度数据

In [1]:
import numpy as np
import pandas as pd

In [5]:
data = pd.Series(np.random.randn(9),index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])
data

a  1   -1.934572
   2   -0.502349
   3    0.370259
b  1   -0.722966
   3   -1.514405
c  1   -0.305325
   2   -0.743389
d  2   -1.569463
   3    0.000860
dtype: float64

看到的结果是经过美化的带有MultiIndex索引的Series的格式。
索引之间的“间隔”表示“直接使⽤上⾯的标签”

In [6]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
#对于⼀个层次化索引的对象，可以使⽤所谓的部分索引，使⽤它选取数据⼦集的操作更简单
data['b']

,0
1,-0.722966
3,-1.514405


In [9]:
data['b':'c']

b  1   -0.722966
   3   -1.514405
c  1   -0.305325
   2   -0.743389
dtype: float64

In [10]:
data.loc[['b','d']]

b  1   -0.722966
   3   -1.514405
d  2   -1.569463
   3    0.000860
dtype: float64

In [8]:
# 甚⾄还可以在“内层”中进⾏选取：
data.loc[:,2]

,0
a,-0.502349
c,-0.743389
d,-1.569463


层次化索引在数据重塑和基于分组的操作（如透视表⽣成）中扮
演着重要的⻆⾊。例如，可以通过unstack⽅法将这段数据重新
安排到⼀个DataFrame中：

In [11]:
data.unstack()

,1,2,3
a,-1.934572,-0.502349,0.370259
b,-0.722966,NaN,-1.514405
c,-0.305325,-0.743389,NaN
d,NaN,-1.569463,0.000860


In [13]:
# unstack的逆运算是stack：
data.unstack().stack()

a  1   -1.934572
   2   -0.502349
   3    0.370259
b  1   -0.722966
   3   -1.514405
c  1   -0.305325
   2   -0.743389
d  2   -1.569463
   3    0.000860
dtype: float64

In [14]:
# 对于⼀个DataFrame，每条轴都可以有分层索引：
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a','a','b','b'],[1,2,1,2]],
                     columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各层都可以有名字（可以是字符串，也可以是别的Python对
象）。如果指定了名称，它们就会显示在控制台输出中：

In [15]:
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
# 有了部分列索引，因此可以轻松选取列分组：
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [19]:
# 可以单独创建MultiIndex然后复⽤。上⾯那个DataFrame中的（带有分级名称）列可以这样创建：
pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],names=['state','color'])


MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

你需要重新调整某条轴上各级别的顺序，或根据指定级别
上的值对数据进⾏排序。swaplevel接受两个级别编号或名称，
并返回⼀个互换了级别的新对象（但数据不会发⽣变化）：

In [20]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

⽽sort_index则根据单个级别中的值对数据进⾏排序。交换级别
时，常常也会⽤到sort_index，这样最终结果就是按照指定顺序
进⾏字⺟排序了：

In [21]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [22]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [27]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

许多对DataFrame和Series的描述和汇总统计都有⼀个level选
项，它⽤于指定在某条轴上求和的级别。再以上⾯那个
DataFrame为例，我们可以根据⾏或列上的级别来进⾏求和：

In [28]:
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [31]:
frame.groupby(level='color', axis=1).sum()

<ipython-input-31-50d8331a41b2>:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level='color', axis=1).sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

使⽤DataFrame的列进⾏索引
⼈们经常想要将DataFrame的⼀个或多个列当做⾏索引来⽤，或
者可能希望将⾏索引变成DataFrame的列。以下⾯这个
DataFrame为例：

In [42]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),
                     'c':['one','one','one','two','two','two','two'],
                     'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [43]:
# DataFrame的set_index函数会将其⼀个或多个列转换为⾏索引，并创建⼀个新的DataFrame：
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [44]:
# 默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来：
frame3 = frame.set_index(['c','d'],drop=False)
frame3

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [45]:
#  reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列⾥⾯：

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


pandas对象中的数据可以通过⼀些⽅式进⾏合并：
pandas.merge可根据⼀个或多个键将不同DataFrame中的⾏
连接起来。SQL或其他关系型数据库的⽤户对此应该会⽐较
熟悉，因为它实现的就是数据库的join操作。
pandas.concat可以沿着⼀条轴将多个对象堆叠到⼀起。
实例⽅法combine_first可以将重复数据编接在⼀起，⽤⼀个对
象中的值填充另⼀个对象中的缺失值。
数据库⻛格的DataFrame合并
数据集的合并（merge）或连接（join）运算是通过⼀个或多个
键将⾏链接起来的。这些运算是关系型数据库（基于SQL）的核
⼼。pandas的merge函数是对数据应⽤这些算法的主要切⼊点。
以⼀个简单的例⼦开始：

In [48]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [49]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


这是⼀种多对⼀的合并。df1中的数据有多个被标记为a和b的
⾏，⽽df2中key列的每个值则仅对应⼀⾏。对这些对象调⽤
merge即可得到：

In [51]:
merged = pd.merge(df1,df2)
merged
# 注意，我并没有指明要⽤哪个列进⾏连接。如果没有指定，
# merge就会将重叠列的列名当做键。不过，最好明确指定⼀下：
# pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


如果两个对象的列名不同，也可以分别进⾏指定：

In [52]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],'data2':range(3)})
mergedd3d4 = pd.merge(df3,df4,left_on='lkey',right_on='rkey')
mergedd3d4

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


可能你已经注意到了，结果⾥⾯c和d以及与之相关的数据消失
了。默认情况下，merge做的是“内连接”；结果中的键是交集。
其他⽅式还有"left"、"right"以及"outer"。外连接求取的是键的并
集，组合了左连接和右连接的效果：

In [53]:
mergedouter = pd.merge(df1,df2,how='outer')
mergedouter

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


多对多的合并有些不直观。看下⾯的例⼦：

In [54]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],'data2':range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [56]:
mergeLeft = pd.merge(df1,df2,on='key',how='left')
mergeLeft

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


多对多连接产⽣的是⾏的笛卡尔积。由于左边的DataFrame有3
个"b"⾏，右边的有2个，所以最终结果中就有6个"b"⾏。连接⽅
式只影响出现在结果中的不同的键的值：

In [57]:
mergeInner = pd.merge(df1,df2,on='key',how='inner')
mergeInner

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


要根据多个键进⾏合并，传⼊⼀个由列名组成的列表即可：

In [58]:
# 定义左侧数据框
left = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1, 2, 3]
})

# 定义右侧数据框
right = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'two', 'two'],
    'rval': [4, 5, 6, 7]
})

# 合并两个数据框
result = pd.merge(left, right, on=['key1', 'key2'], how='outer')

# 打印结果
print(result)


  key1 key2  lval  rval
0  bar  one   3.0   NaN
1  bar  two   NaN   6.0
2  bar  two   NaN   7.0
3  foo  one   1.0   4.0
4  foo  one   1.0   5.0
5  foo  two   2.0   NaN


结果中会出现哪些键组合取决于所选的合并⽅式，你可以这样来
理解：多个键形成⼀系列元组，并将其当做单个连接键（当然，
实际上并不是这么回事）。
注意：在进⾏列－列连接时，DataFrame对象中的索引会被
丢弃。

对于合并运算需要考虑的最后⼀个问题是对重复列名的处理。虽
然你可以⼿⼯处理列名重叠的问题（查看前⾯介绍的重命名轴标
签），但merge有⼀个更实⽤的suffixes选项，⽤于指定附加到左
右两个DataFrame对象的重叠列名上的字符串

In [60]:
mergeByKey1 = pd.merge(left,right,on='key1')
mergeByKey1

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,two,6
5,bar,one,3,two,7


In [61]:
# suffixes 是什么？
# 在 Pandas 的 merge 方法中，suffixes 参数用于处理 左右两张表中存在相同列名时 的情况。
# 通过 suffixes，可以为重复的列名添加后缀，以便区分它们属于哪张表。
suffixes = pd.merge(left, right, on='key1', suffixes=('_left', '_right'))
suffixes

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,two,6
5,bar,one,3,two,7


有时候，DataFrame中的连接键位于其索引中。在这种情况下，
你可以传⼊left_index=True或right_index=True（或两个都传）
以说明索引应该被⽤作连接键：

In [63]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])
result1 = pd.merge(left1, right1, left_on='key', right_index=True)
result1

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


由于默认的merge⽅法是求取连接键的交集，因此你可以通过外
连接的⽅式得到它们的并集：

In [64]:
result2 = pd.merge(left1,right1,left_on='key',right_index=True,how='outer')
result2

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


对于层次化索引的数据，事情就有点复杂了，因为索引的合并默
认是多键合并：

In [65]:
lefth = pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],'key2':[2000,2001,2002,2001,2002],'data':np.arange(5.)})
righth = pd.DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
           [2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2']
)

这种情况下，你必须以列表的形式指明⽤作合并键的多个列（注
意⽤how='outer'对重复索引值的处理）：

In [66]:
result3 = pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)
result3

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [69]:
result4 = pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')
result4

,key1,key2,data,event1,event2
4,Nevada,2000,NaN,2.0,3.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0


同时使⽤合并双⽅的索引也没问题：

In [70]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],index=['a','c','e'],columns=['Ohio','Nevada'])
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],index=['b','c','d','e'],columns=['Missouri','Alabama'])
result5 = pd.merge(left2,right2,how='outer',left_index=True,right_index=True)
# 也就是两边都把自己的索引当做key来outer合并
result5

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame还有⼀个便捷的join实例⽅法，它能更为⽅便地实现
按索引合并。它还可⽤于合并多个带有相同或相似索引的
DataFrame对象，但要求没有重叠的列。在上⾯那个例⼦中，我
们可以编写：

In [71]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


最后，对于简单的索引合并，你还可以向join传⼊⼀组
DataFrame，下⼀节会介绍更为通⽤的concat函数，也能实现此
功能

In [73]:
another=pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],index=['a','c','e','f'],columns=['New York','Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [74]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [75]:
left2.join([right2,another],how='outer') #P318

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0
